In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import time
import changefont as cf

import os 
os.chdir(os.getcwd())
import sys
# insert at 1, 0 is the script path (or '' in REPL)
pydice_folder = os.path.dirname(os.getcwd())+"\\1_Model"
policy_folder = os.path.dirname(os.getcwd())+"\\5_Policy_Discovery"
sys.path.insert(1, pydice_folder)

from specify import specify_levers

from ema_workbench.analysis import parcoords
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench import (save_results, load_results, Model, Policy, Scenario, 
                           Constraint, RealParameter, IntegerParameter, ScalarOutcome, 
                           MultiprocessingEvaluator, SequentialEvaluator)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)
BaseEvaluator.reporting_frequency = 0.1
# ema_logging.log_to_stderr(ema_logging.DEBUG)

from PyDICE_SCC import PyDICE_SCC

C:\Users\Shajee\Anaconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)
C:\Users\Shajee\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Shajee\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Shajee\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dice_opt = pd.read_excel("DICE2013R.xlsm" ,sheet_name = "Opttax", index_col = 0)
policy_12 = pd.read_csv("policy_12.csv", header=None, index_col=0).T

In [3]:
nord_optimal_policy = Policy('nord_optimal_policy', **specify_levers(np.mean(dice_opt.iloc[129]),
                                                                     0.015,0,29))

robust_policy = Policy('robust_policy', **specify_levers(policy_12.to_numpy()[0][0],
                                                         policy_12.to_numpy()[0][1],0,
                                                         int(policy_12.to_numpy()[0][2])))                                      

In [4]:
policies_to_evaluate = []
policies_to_evaluate.append(nord_optimal_policy)
policies_to_evaluate.append(robust_policy)

In [5]:
model = PyDICE_SCC()
pydice = Model('PyDICE', function = model)

In [6]:
policies_to_evaluate

[Policy({'sr': 0.24868248835166676, 'irstp': 0.015, 'periodfullpart': 0, 'miu_period': 29}),
 Policy({'sr': 0.2766788046488994, 'irstp': 0.002261578895469605, 'periodfullpart': 0, 'miu_period': 10})]

In [7]:
pydice.uncertainties = [IntegerParameter('t2xco2_index', 0, 999),
                         IntegerParameter('t2xco2_dist',0,2),
                         IntegerParameter('fdamage',0,2),
                         RealParameter('tfp_gr',  0.07, 0.09),
                         RealParameter('sigma_gr', -0.012, -0.008),
                         RealParameter('pop_gr', 0.1, 0.15),
                         RealParameter('fosslim',  4000.0, 13649),
                         IntegerParameter('cback', 100, 600)]

pydice.levers = [RealParameter('sr', 0.1, 0.5),
                  RealParameter('irstp',  0.001, 0.015),
                  IntegerParameter('periodfullpart', 10, 58),
                  IntegerParameter('miu_period', 10, 58)]

pydice.outcomes = [ScalarOutcome('SCC 2015', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2020', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2025', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2030', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2050', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2100', ScalarOutcome.INFO),
                    ScalarOutcome('SCC 2150', ScalarOutcome.INFO)]

In [ ]:
n_scenarios=100000

start = time.time()
with MultiprocessingEvaluator(pydice) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies_to_evaluate)
end = time.time()

print('Experiment time is ' + str(round((end - start)/60)) + ' mintues')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 100000 scenarios * 2 policies * 1 model(s) = 200000 experiments


In [ ]:
experiments, outcomes = results

In [ ]:
outDF = pd.DataFrame(outcomes)

In [ ]:
resDF = pd.concat([experiments, outDF], axis=1)

In [ ]:
resDF[resDF["policy"]=="robust_policy"].iloc[:,15:].describe()

In [ ]:
resDF[resDF["policy"]=="nord_optimal_policy"].iloc[:,15:].describe()

In [1]:
import matplotlib.pyplot as plt

In [ ]:
plt.lay